In [1]:
import loaders.book_content_loader as bcl

In [2]:
res_search = bcl.search_books("Frankenstein Mary Shelley")

In [3]:
url_archive = bcl.get_book_archive_page(res_search)

In [4]:
book_text = bcl.fetch_book_text(url_archive)

Book found with length: 529209 characters
ustic 
guardians  to  yield  their  charge  to  her.  They  were  fond  of 
the  sweet  orphan.  Her  presence  had  seemed  a  blessing  to 
them ;  but  it  would  be  unfair  to  her  to  keep  her
